In [147]:
import cv2
import numpy as np
import PIL
from PIL import Image
import os
from matplotlib import pyplot as plt
import imutils
from sklearn import preprocessing, svm
from skimage.filters import laplace
from scipy.ndimage import variance

In [76]:
# path = "archive/"

In [4]:
# image = Image.open(path+ '0_IPHONE-SE_M.jpg') 
  
# # summarize some details about the image 
# print(image.format) 
# print(image.size) 
# print(image.mode)

JPEG
(1536, 2048)
RGB


In [5]:
# image_arr = np.asarray(image)
# print(type(image_arr))
# print(image_arr.shape)

<class 'numpy.ndarray'>
(2048, 1536, 3)


In [70]:
# list = os.listdir(path)
# p = 0

# for file in list:
#     if p>5:
#         break
#     p += 1
    
# def custom_kernel(img):
#     #img= path+file
#     imag = cv2.imread(img)
#     imag = imutils.resize(imag, width=500)
#     image = increase_brightness(imag)
#     #image=np.uint8(image)
#     height, width, _ = image.shape
#     #print(height, width)
#     print(file)
#     print("-----------------------------------")
#     for i in range(3):
#         for j in range(3):
            
#             x1= int(height*i/4)
#             x2= int(height/2 + height*i/4)
#             y1= int(width*j/4)
#             y2= int(width/2 + width*j/4)
#             #print(x1, x2, y1, y2, end="|||")
#             im = image[y1:y2, x1:x2]

#             gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

#             (mean, blurry) = detect_blur(gray, radius=60)
#             print(blurry, end=" ") 
#             window = 'image'
#             cv2.imshow(window, gray)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows() 
#     print(" ")
#     return blurry

In [148]:
def detect_blur(image, radius=60, threshold=5, vis=False):
    (h, w) = image.shape
    x, y = (int(w/2.0), int(h/2.0))
    
    fft = np.fft.fft2(image)
    fft_sh = np.fft.fftshift(fft)
    
    fft_sh[y-radius:y+radius, x-radius:x+radius] = 0
    fft_sh = np.fft.ifftshift(fft_sh)
    recon = np.fft.ifft2(fft_sh)
    
    mag = 20*np.log(np.abs(recon))
    mean = np.mean(mag)
    
    return (mean, mean<=threshold)

In [149]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [123]:
# def make_dataset(train_images):
#     x=[]
#     y=[]
    
#     i=1
    
#     for image in train_images:
#         #print(i, image)
#         i += 1
#         img = cv2.imread(image)
#         img = imutils.resize(img, width=500)
#         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         x.append(gray)
        
#         if "und" in image:
#             y.append(0)
#         else:
#             y.append(1)
#     return x,y

In [124]:
# X, Y = make_dataset(train_images)
# print("Done")

1 archive/sharp/und (104).JPG
2 archive/sharp/und (224).jpg
3 archive/defocused_blurred/181_HONOR-10_F.jpg
4 archive/defocused_blurred/276_NIKON-D3400-18-55MM_F.JPG
5 archive/sharp/und (114).jpg
6 archive/motion_blurred/106_NIKON-D3400-35MM_M.JPG
7 archive/sharp/und (3).jpeg
8 archive/defocused_blurred/186_HONOR-8_F.jpg
9 archive/motion_blurred/47_XIAOMI-REDMI-7_M.jpg
10 archive/defocused_blurred/116_BQ-5512L_F.jpg
11 archive/motion_blurred/290_IPHONE-SE_M.jpg
12 archive/motion_blurred/307_HONOR-7X_M.jpg
13 archive/motion_blurred/51_IPHONE-SE_M.jpg
14 archive/motion_blurred/212_IPHONE-SE_M.jpg
15 archive/defocused_blurred/325_SAMSUNG-GALAXY-J3_F.jpg
16 archive/sharp/und (179).jpg
17 archive/defocused_blurred/33_HONOR-8X_F.jpg
18 archive/defocused_blurred/193_IPHONE-7_F.jpeg
19 archive/defocused_blurred/264_IPHONE-7_F.jpeg
20 archive/defocused_blurred/32_HUAWEI-P-SMART_F.jpg
21 archive/defocused_blurred/167_HONOR-7X_F.jpg
22 archive/motion_blurred/85_HONOR-8X_M.jpg
23 archive/defocused_

In [150]:
def laplace_values(img):
    img = cv2.imread(img)
    img = imutils.resize(img, width=500)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edge_laplace = laplace(gray, ksize=3)
    var = variance(edge_laplace)
    maximum = np.amax(edge_laplace)
    return var, maximum

In [151]:
def custom_kernel(img):
    img = cv2.imread(img)
    img = imutils.resize(img, width=500)
    img = increase_brightness(img)
    height, width, _ = img.shape
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    (mean, blurry) = detect_blur(gray, radius=60)
    return mean, blurry

In [152]:
def train_x(img):
    var, maximum = laplace_values(img)
    mean, blur = custom_kernel(img)
    x_train = ([var, maximum, mean, blur])
    return x_train

## Making dataset

In [153]:
trainx = []
y = []
for x in train_images:
    trainx.append(train_x(x))
    if "und" in x:
        y.append(0)
    else:
        y.append(1)
print("Done")

Done


## Splitting dataset

In [154]:
X = np.array(trainx)
Y = np.array(Y)
xtrain = X[:700]
xtest = X[700:]
ytrain = Y[:700]
ytest = Y[700:]

print(xtest)

[[ 1.00338037e-02  8.23529412e-01  2.60454085e+01  0.00000000e+00]
 [ 3.11202356e-03  8.00000000e-01  1.99928787e+01  0.00000000e+00]
 [ 1.84241370e-04  1.09803922e-01  5.15791869e-01  1.00000000e+00]
 ...
 [ 2.78603524e-04  1.80392157e-01 -2.88895406e+00  1.00000000e+00]
 [ 1.70617798e-02  1.73725490e+00  2.81584219e+01  0.00000000e+00]
 [ 1.62994311e-04  7.45098039e-02 -1.03550198e+01  1.00000000e+00]]


## Training and testing

In [155]:
clf = svm.SVC(kernel="linear")
clf.fit(xtrain,ytrain)
clf.score(xtest, ytest)

0.9028571428571428

In [191]:
def test_image(img):
    print(img)
    var, maximum = laplace_values(img)
    mean, blur = custom_kernel(img)
    test = ([var, maximum, mean, blur])
    test = np.array(test)
    #print(test)
    x = clf.predict((test,))
    
    #print(x)
    if x==0:
        print("Not Blurry")
    else:
        print("Blurry")

In [192]:
#change path for trying different images
path = "archive/motion_blurred/1_XIAOMI-PROCOFONE-F1_M.jpg"
test_image(path)

archive/motion_blurred/1_XIAOMI-PROCOFONE-F1_M.jpg
Blurry
